## Example 1 problem from [Lakhdar et al. (2005)](http://onlinelibrary.wiley.com/doi/10.1021/bp0501571/abstract)

$\text{Maximise}$

$$\text{Profit} = \sum_p \sum_t (\nu_pS_{pt} - \sum_i \eta_pCP_{ipt} - \sum_j \eta_pFP_{jpt} - \sum_i \psi_pZ_{ipt} - \sum_j \psi_pX_{jpt} - \rho_pCI_{pt} - \omega_pFI_{pt} - \delta_p\Delta_{pt} - \tau_p(CW_{pt} + FW_{pt}))$$

$\text{Production constraints}$

$$CP_{ipt} = Z_{ipt} + CR_p (CT_{ipt} - \alpha_p Z_{ipt}) \quad \forall i, p, t$$
<br>
$$FP_{jpt} = W_{jpt} + FR_p (FT_{jpt} - \beta_p W_{jpt}) \quad \forall j, p, t$$
<br>
$$Z_{ipt} \ge Y_{ipt} - Y_{ip,t-1} \quad \forall i, p, t$$
<br>
$$X_{jpt} \ge U_{jpt} - U_{jp,t-1} \quad \forall j, p, t$$
<br>
$$W_{jpt} \ge \frac{\sum_i Z_{ipt}} {\big|i\big|} + X_{jpt} - 1 \quad \forall j, p, t$$ 
<br>
$$\sum_p Y_{ipt} \le 1 \quad \forall i, t$$
<br>
$$\sum_p U_{jpt} \le 1 \quad \forall j, t$$

$\text{Timing constraints}$

$$CT_p^{min} Y_{ipt} \le CT_{ipt} \quad \forall i, p, t$$
<br>
$$CT_p^{max} Y_{ipt} \ge CT_{ipt} \quad \forall i, p, t$$
<br>
$$FT_p^{min} U_{jpt} \le FT_{jpt} \quad \forall j, p, t$$
<br>
$$FT_p^{max} U_{jpt} \ge FT_{jpt} \quad \forall j, p, t$$
<br>
$$CT_{it}^{tot} = \sum_p CT_{ipt} \quad \forall i, t$$
<br>
$$FT_{jt}^{tot} = \sum_p FT_{jpt} \quad \forall j, t$$
<br>
$$CT_{it}^{tot} \le H_t \quad \forall i, t$$
<br>
$$FT_{jt}^{tot} \le H_t \quad \forall j, t$$

$\text{Storage constraints}$

$$CI_{pt} = CI_{p,t-1} + \sum_i CP_{ipt} - \frac{1}{\lambda_p} \sum_j FP_{jpt} - CW_{pt} \quad \forall p, t$$
<br>
$$FI_{pt} = FI_{p,t-1} + \sum_j FP_{jpt} - S_{pt} - FW_{pt} \quad \forall p, t$$
<br>
$$0 \le CI_{pt} \le C_p \quad \forall p, t$$
<br>
$$0 \le FI_{pt} \le F_p \quad \forall p, t$$
<br>
$$CI_{pt} \le \sum_j \sum_{\theta=t+1}^{t+\zeta_p} FP_{jp\theta} \quad \forall p, t$$
<br>
$$FI_{pt} \le \sum_{\theta=t+1}^{t+\sigma_p} S_{p\theta} \quad \forall p, t$$

$\text{Sales and penalty constraint}$

$$\Delta_{pt} = \Delta_{p,t-1} + D_{pt} - S_{pt} \quad \forall p, t$$

### Input data

In [ ]:
num_products = 3 # |p|
num_usp_suites = 2 # |i|
num_dsp_suites = 2 # |j|
num_periods = 6 # |t|

demand = [0 0 0 6 0 6;
          0 0 6 0 0 0;
          0 8 0 0 8 0]

# USP parameters
CR = [0.05 0.045 0.08]
alpha = [30 32 22.5]
zeta = [1 1 1]
C = [10 10 10]
CTmin = [20 21 12.5]
CTmax = [60 60 60]
rho = [5 5 5]
nu = [20 20 20]
eta = [2 2 2]
tau = [5 5 5]

# DSP parameters
FR = [0.1 0.1 0.1]
beta = [40 42 34.5]
sigma = [3 3 3]
F = [40 40 40]
FTmin = [10 10 10]
FTmax = [60 60 60]
omega = [1 1 1]

# Other
delta = [20 20 20]
psi = [1 1 1]
lambda = [1 1 1]
H = 60

### Implementation

#### Variables

In [32]:
using JuMP, CPLEX

model = Model(solver=CplexSolver())

# Binary variables
@variable(model, U[j=1:num_dsp_suites, p=1:num_products, t=1:num_periods], Bin)
@variable(model, W[j=1:num_dsp_suites, p=1:num_products, t=1:num_periods], Bin)
@variable(model, X[j=1:num_dsp_suites, p=1:num_products, t=1:num_periods], Bin)
@variable(model, Y[i=1:num_usp_suites, p=1:num_products, t=1:num_periods], Bin)
@variable(model, Z[i=1:num_usp_suites, p=1:num_products, t=1:num_periods], Bin)

# Integer variables
@variable(model, CP[i=1:num_usp_suites, p=1:num_products, t=1:num_periods] >= 0, Int)
@variable(model, CI[p=1:num_products, t=1:num_periods] >= 0, Int)
@variable(model, CW[p=1:num_products, t=1:num_periods] >= 0, Int)
@variable(model, FP[j=1:num_dsp_suites, p=1:num_products, t=1:num_periods] >= 0, Int)
@variable(model, FI[p=1:num_products, t=1:num_periods] >= 0, Int)
@variable(model, FW[p=1:num_products, t=1:num_periods] >= 0, Int)
@variable(model, S[p=1:num_products, t=1:num_periods] >= 0, Int)
@variable(model, Delta[p=1:num_products, t=1:num_periods] >= 0, Int)

# Continuous variables
@variable(model, CT[i=1:num_usp_suites, p=1:num_products, t=1:num_periods] >= 0)
@variable(model, CTtot[i=1:num_usp_suites, t=1:num_periods] >= 0)
@variable(model, FT[j=1:num_dsp_suites, p=1:num_products, t=1:num_periods] >= 0)
@variable(model, FTtot[j=1:num_dsp_suites, t=1:num_periods] >= 0)

model

Feasibility problem with:
 * 0 linear constraints
 * 456 variables: 180 binary, 180 integer
Solver is Cplex

#### Production constraints

In [33]:
for i=1:num_usp_suites, p=1:num_products, t=1:num_periods 
    @constraint(model, CP[i,p,t] == Z[i,p,t] + 
                        CR[p]*(CT[i,p,t] - alpha[p]*Z[i,p,t]))
end

for j=1:num_dsp_suites, p=1:num_products, t=1:num_periods 
    @constraint(model, FP[j,p,t] == W[j,p,t] + 
                        FR[p]*(FT[j,p,t] - beta[p]*W[j,p,t]))
end

for i=1:num_usp_suites, p=1:num_products
    @constraint(model, Z[i,p,1] >= Y[i,p,1])
end

for i=1:num_usp_suites, p=1:num_products, t=2:num_periods 
    @constraint(model, Z[i,p,t] >= Y[i,p,t] - Y[i,p,t-1])
end

for j=1:num_dsp_suites, p=1:num_products
    @constraint(model, X[j,p,1] >= U[j,p,1])
end

for j=1:num_dsp_suites, p=1:num_products, t=2:num_periods 
    @constraint(model, X[j,p,t] >= U[j,p,t] - U[j,p,t-1])
end

for j=1:num_dsp_suites, p=1:num_products, t=1:num_periods 
    @constraint(model, W[j,p,t] >= (sum(Z[i,p,t] for i=1:num_usp_suites)
                                    / num_usp_suites) + X[j,p,t] - 1) 
end

for i=1:num_usp_suites, t=1:num_periods
    @constraint(model, sum(Y[i,p,t] for p=1:num_products) <= 1)
end

for j=1:num_dsp_suites, t=1:num_periods
    @constraint(model, sum(U[j,p,t] for p=1:num_products) <= 1)
end

#### Timing constraints

In [34]:
for i=1:num_usp_suites, p=1:num_products, t=1:num_periods
    @constraint(model, CTmin[p] * Y[i,p,t] <= CT[i,p,t])
end

for i=1:num_usp_suites, p=1:num_products, t=1:num_periods
    @constraint(model, CTmax[p] * Y[i,p,t] >= CT[i,p,t])  
end

for j=1:num_dsp_suites, p=1:num_products, t=1:num_periods
    @constraint(model, FTmin[p] * U[j,p,t] <= FT[j,p,t])
end

for j=1:num_dsp_suites, p=1:num_products, t=1:num_periods
    @constraint(model, FTmax[p] * U[j,p,t] >= FT[j,p,t]) 
end

for i=1:num_usp_suites, t=1:num_periods
    @constraint(model, CTtot[i,t] == sum(CT[i,p,t] for p=1:num_products))
end

for i=1:num_usp_suites, t=1:num_periods
    @constraint(model, CTtot[i,t] <= H)
end

for j=1:num_dsp_suites, t=1:num_periods
    @constraint(model, FTtot[j,t] == sum(FT[j,p,t] for p=1:num_products))  
end

for j=1:num_dsp_suites, t=1:num_periods
    @constraint(model, FTtot[j,t] <= H)  
end

#### Storage constraints

In [35]:
for p=1:num_products
    @constraint(model, CI[p,1] == sum(CP[i,p,1] for i=1:num_usp_suites) 
                                  - (1/lambda[p])*sum(FP[j,p,1] for j=1:num_dsp_suites) - CW[p,1]) 
end

for p=1:num_products, t=2:num_periods
    @constraint(model, CI[p,t] == CI[p,t-1] + sum(CP[i,p,t] for i=1:num_usp_suites) 
                                  - (1/lambda[p])*sum(FP[j,p,t] for j=1:num_dsp_suites) - CW[p,t]) 
end

for p=1:num_products
    @constraint(model, FI[p,1] == sum(FP[j,p,1] for j=1:num_dsp_suites) - S[p,1] - FW[p,1])
end

for p=1:num_products, t=2:num_periods
    @constraint(model, FI[p,t] == FI[p,t-1] + sum(FP[j,p,t] for j=1:num_dsp_suites) - S[p,t] - FW[p,t])
end

for p=1:num_products, t=1:num_periods
    @constraint(model, CI[p,t] <= C[p])
end

for p=1:num_products, t=1:num_periods
    @constraint(model, FI[p,t] <= F[p]) 
end

for p=1:num_products, t=1:num_periods
    @constraint(model, CI[p,t] <= sum(FP[j,p,theta] for j=1:num_dsp_suites, theta=(t+1):(t+zeta[p]) if theta <= num_periods))                                
end

for p=1:num_products, t=1:num_periods
    @constraint(model, FI[p,t] <= sum(S[p,theta] for theta=(t+1):(t+sigma[p]) if theta <= num_periods))
end

#### Sales and penalty constraint

In [36]:
for p=1:num_products
    @constraint(model, Delta[p,1] == demand[p,1] - S[p,1]) 
end

for p=1:num_products, t=2:num_periods
   @constraint(model, Delta[p,t] == Delta[p,t-1] + demand[p,t] - S[p,t]) 
end

In [37]:
#print(model)

#### Objective

In [38]:
@objective(model, Max, 
           sum(nu[p]*S[p,t] 
               - sum(eta[p]*CP[i,p,t] for i=1:num_usp_suites)
               - sum(eta[p]*FP[j,p,t] for j=1:num_dsp_suites)
               - sum(psi[p]*Z[i,p,t] for i=1:num_usp_suites)
               - sum(psi[p]*X[j,p,t] for j=1:num_dsp_suites)
               - rho[p]*CI[p,t]
               - omega[p]*FI[p,t]
               - delta[p]*Delta[p,t]
               - tau[p]*(CW[p,t] + FW[p,t])
               for p=1:num_products, t=1:num_periods))

status = solve(model)


Root node processing (before b&c):
  Real time             =    0.00 sec. (0.60 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.60 ticks)
CPLEX Error  1217: No solution exists.
Tried aggregator 2 times.
MIP Presolve eliminated 74 rows and 44 columns.
MIP Presolve modified 98 coefficients.
Aggregator did 25 substitutions.
Reduced MIP has 423 rows, 387 columns, and 1293 nonzeros.
Reduced MIP has 172 binaries, 155 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.40 ticks)
Found incumbent of value -2080.000000 after 0.00 sec. (1.74 ticks)
Probing fixed 0 vars, tightened 18 bounds.
Probing changed sense of 4 constraints.
Probing time = 0.00 sec. (0.41 ticks)
Cover probing fixed 0 vars, tightened 8 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 4 columns.
MIP Pre

:Optimal